In [1]:
# Optional: change Jupyter Notebook theme to GDD theme
from IPython.core.display import HTML
HTML(url='https://gdd.li/jupyter-theme')

![footer_logo](https://marysia.nl/assets/GDD/css/logo.png)

# Ceteris Paribus Plots

In this notebook, we shall take our first steps towards *model-agnostic* interpretability methods. That is, methods of interpretability that are not reliant on the algorithm's innate interpretability. They can be used on any model, regardless of how easy the model is to understand by analysing the algorithm alone. 

We will address Ceteris Paribus plots, both on the Support Vector Classifier model we saw in the previous notebook, as well as on a new problem -- predicting the body mass of a given penguin, given its bodily measurements, sex and species. 

### Outline
1. [Create the model](#model)
1. [Ceteris Paribus](#cetpar)
1. [Exercise 1](#ex1)
1. [Regression & Categorical features](#regcat)
1. [Exercise 2](#ex2)

![](https://github.com/allisonhorst/palmerpenguins/raw/master/man/figures/logo.png)

<a id = 'model'></a>

# 1. Create the Model

We start out by again loading in the data. This is the same as we saw in the previous notebook -- we load in the penguins dataset, drop the Adelie penguin for simplicity's sake, define the features (flipper length and body mass) and target (species). Based on that, we can create our feature matrix X and target vector y and split it into a train & test part. 


In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split

# Load in the data. 
penguins = (
    pd.read_csv('data/penguins.csv')
    .loc[lambda d: d['species'] != 'Adelie']
    .dropna()
)

# Define features & target
feature_columns = ['flipper_length_mm', 'body_mass_g']
target = 'species'

# Set X and y
X = penguins.loc[:, feature_columns]
y = penguins.loc[:, target]

# Split the dataset. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

However, we need one small change compared to our previous workflow -- scikit-learn is able to handle both numeric input (integers) as well as strings (words) for the target variable. That meant that, in our previous notebook, we could keep the target variable as Chinstrap or Gentoo for each row. 

However, the explainability package that we will be using later on this notebook requires our target to be numeric -- so 0 and 1 instead of Chinstrap and Gentoo. Therefore, we must also train our model -- the model that is to be explained by our explainability package -- on numeric targets. In order to do this, we simply create a mapping where Chinstraps are to be represented with a zero and Gentoo penguins with a one. 

In [ ]:
mapping = {'Chinstrap': 0, 'Gentoo': 1}
y_train = y_train.replace(mapping)
y_test = y_test.replace(mapping)

Other than changing the representation of our target variable, everything else related to training the model remains the same. We import the support vector classifier, instantiate it, fit it on our train set, and review the accuracy on the test set. 

In [ ]:
from sklearn.svm import SVC

model_svm = SVC()
model_svm.fit(X_train, y_train)
model_svm.score(X_test, y_test)

Great! Even though we changed our y_train and y_test representation, the score remains the same. Now let's get some insights.

<a id = 'cetpar'></a>

# 2. Ceteris Paribus Plots

Let us grab our data point from the previous notebook, and see if we can find some kind of explanation of what would be needed to change this datapoint.

In [ ]:
flipper_length_mm = 195
body_mass_g = 4400

example_datapoint = pd.DataFrame.from_dict({
    'flipper_length_mm': [flipper_length_mm], 
    'body_mass_g': [body_mass_g]
})


![footer_logo](https://dalex.drwhy.ai/misc/dalex_even.png)

Our SVM model in itself is not very interpretable. That is why we need an external tool.

`dalex`, short for _moDel Agnostic Language for Exploration and eXplanation_, is a package for both Python and R that is part of the [DrWhy.AI](https://github.com/ModelOriented/DrWhy/blob/master/README.md) universe. DrWhy.AI concerns itself with the question "How to develop machine learning models in a responsible manner?" and one of the areas it focuses on is _transparency_. Does the user know what influences model predictions? If the model decisions affect us directly or indirectly, we should know where these decisions come from and how they can be changed. And that's where dalex comes in. 

First of all, we need to import the package and create an _Explainer_.

In [ ]:
import dalex

explainer = dalex.Explainer(model_svm, X_train, y_train)

The Explainer object that we just created, creates a wrapper around a predictive model. Black-box models may have very different structures. Right now, we are using dalex with sklearn, but we may - for example - also use it with a Tensorflow model. 

The Explainer class creates a unified representation of a model, which can then be used or processed to produce various types of explainations that can be plotted. 

This means we pass it the model to be explained. Depending on the type of explanations that we want, we also pass it the data it was trained on -- X_train and y_train. This is not always necessary, but it is now.

In [ ]:
profile = explainer.predict_profile(example_datapoint)

Once we have our explainer, we can create a _profile_ given our single example datapoint. There are several types of profiles that we can create, but the default (and the one we will focus on for now) is _partial_. As you can see, it calculates the _ceteris paribus_ profile. 

Once the profile is created, we can easily plot it. 

In [ ]:
profile.plot()

<a id = 'ex1'></a>
# 3. Exercise
<div class="exercise" markdown="1">

### Exercise 
#### Ceteris Paribus plots for classification

Analyse the ceteris paribus plots of the two features created with dalex.

1. What do you notice? Do these plots help you discover something interesting about the model?
1. Flip the prediction: analyse the ceteris paribus plot and determine what values (flipper length and/or body mass) must be changed and how much in order to flip the prediction. **Note**: 0 is Chinstrap, 1 is Gentoo!
1. Do you have an intuition on how the values are calculated? Do you think you would be able to reproduce one of the values, for instance, the prediction for body_mass_g = 4000? 
1. Describe in your own words what the ceteris paribus plots display.
</div>


In [ ]:
# Change the values to change the prediction. 
flipper_length_mm = 195
body_mass_g = 4400

example_datapoint = pd.DataFrame.from_dict({
    'flipper_length_mm': [flipper_length_mm], 
    'body_mass_g': [body_mass_g]
})

svm_pred = model_svm.predict(example_datapoint)

print(f'Prediction SVM: {svm_pred[0]}')


<a id = 'regcat'></a>

# 4. Regression & Categorical Features
![footer_logo](https://github.com/allisonhorst/palmerpenguins/raw/master/man/figures/lter_penguins.png) 


Our initial problem was relatively simple: predict the penguin species (Gentoo or Chinstrap) based on two features. Let's investigate a slightly more complex problem instead, and see how Ceteris Paribus plots can help us understand what's going on. 

We are going to redefine our feature matrix X and target vector y to create a _regression_ problem. That is, a problem for which the outcome is numeric, rather than a category. Our target will be to predict _a given penguin's body mass in grams_. 

In addition, we will use a combination of _numeric_ and _categorical_ features, and include all three penguin species -- including the Adélie penguin! 


In [ ]:
# Read in the data again - do *not* drop Adélie! 
penguins = (
    pd.read_csv('data/penguins.csv')
    .dropna()
)

# Set features & target.
feature_columns = ['flipper_length_mm', 'bill_length_mm', 'bill_depth_mm', 'sex', 'island', 'species']
target = 'body_mass_g'

# Set X and y
X = penguins.loc[:, feature_columns]
y = penguins.loc[:, target]

# Split the dataset. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

Creating our model becomes a little bit more complicated, because we need a way to process the newly introduced _categorical_ features (the penguin's sex, the island it was sighted on, and the species) to a numeric format. This is because scikit-learn expects all its input to be numeric. The way we choose to process this is with _one-hot encoding_.

One-hot encoding -- also known as dummy encoding -- entails adding binary variables for every unique category. In the case of the *island* column, this means 3 extra columns will be created because there are 3 possible islands. 

In this case, we use ColumnTransformer to encode our data, because we can then indicate which columns we want to one-hot encode (only the categorical ones). 

Next, we create a pipeline containing two steps: the ColumnTransformer we just created and the model we want to use -- in this case a random forst regressor. The beauty of pipelines is that they can be treated like any other model, which means we can plug it in anywhere we would an estimator!

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

ct = ColumnTransformer([
    ('onehot', OneHotEncoder(), ['sex', 'species', 'island']),
], remainder='passthrough')

pipeline = Pipeline([
    ('ct', ct),
    ('model', RandomForestRegressor())
])

pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

Now that we have our trained model, in the form of a sklearn pipeline, we can again create an Explainer object for this model. Why do we have to create a new object and can we not use the previously created `explainer` variable? Because each Explainer that is created is specific to the model/algorithm that was trained.

In [ ]:
explainer = dalex.Explainer(pipeline, X_train, y_train)

Let us now at random choose a row from our test set to understand. Feel free to adjust it to pick another row! 

In [ ]:
n = 230
row = X_test.loc[[230]]
row

In [ ]:
pipeline.predict(row)

We now again calculate a ceteris paribus profile for the chosen row, and immediately plot the profile to show how flipper length, bill length and bill depth influence the prediction. 

In [ ]:
profile = explainer.predict_profile(row)
profile.plot()

However, these are just the _numeric_ features. What about the categorical ones? 

We can investigate the categorical features by adding an argument to the plotting method. Notice that we do not have to re-calculate the profile! 

In [ ]:
profile.plot(variable_type="categorical")

<a id = 'ex2'></a>
# 5. Exercise
<div class="exercise" markdown="1">

### Exercise 
#### Ceteris Paribus plots

Analyse the ceteris paribus plots of the numeric and categorical features created with dalex.

1. Look at the categorical features. What features and associated values increase the model's prediction of body weight? Which ones decrease the model's prediction? Which features hardly have any impact? 
2. Can you reproduce one of the graphs? E.g. the predictions for the three possible species. 
3. Execute the cells below with the linear regression model. Compare the Ceteris Paribus plots of the linear regression model to the random forest regressor. Can you explain why they look different? 
4. Compare the Ceteris Paribus plots for the Island variable for the Random Forest regressor and the Linear Regression model. Are they the same or different? Can you explain why?
5. Can you think of any circumstances in which a Ceteris Paribus plot would be helpful? Can you think of any downsides? Write down advantages and disadvantages.

**Bonus**: Go back a few cells and choose a different value for _n_ (the row to choose). Rerun the plots. Do they look the same or different? _Hint:_ use `X_test.sample()` to take a random data point in the test set, and set _n_ to the value in the index.
</div>


In [ ]:
X_test.sample()

Additional code for the exercise: 

In this case, we are creating a new model -- this time not a Random Forest regressor, but a linear regression model. Notice how we rename the pipeline variable to `pipeline_lr` to denote it is our linear regression model pipeline.

In [ ]:
from sklearn.linear_model import LinearRegression 

ct = ColumnTransformer([
    ('onehot', OneHotEncoder(), ['sex', 'species', 'island']),
], remainder='passthrough')

pipeline_lr = Pipeline([
    ('ct', ct),
    ('model', LinearRegression())
])

pipeline_lr.fit(X_train, y_train)
pipeline_lr.score(X_test, y_test)

We must create a new explainer as well, one for our linear regression pipeline. Notice how we do not overwrite the original explainer, but create one specifically for our linear regression model called `explainer_lr`.

In [ ]:
explainer_lr = dalex.Explainer(pipeline_lr, X_train, y_train)

Create and plot a profile for our linear regression pipeline.

In [ ]:
profile_lr = explainer_lr.predict_profile(row)
profile_lr.plot()

In [ ]:
profile_lr.plot(variable_type="categorical")

------